## Imports

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Flatten, Conv1D, MaxPooling1D, Dense
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
import time
import numpy as np
import pickle

## Set Memory Growth True

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Loading Training Data

In [ ]:
pickle_in = open("../../assets/audio_sentiment_data_v1/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("../../assets/audio_sentiment_data_v1/y.pickle","rb")
y = pickle.load(pickle_in)

In [ ]:
X=np.reshape(X,(X.shape[0],26,1))

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
paths = {"checkpath" : "../../assets/audio_sentiment_data_v1/checkpoints",
         "modelpath" : "../../assets/audio_sentiment_data_v1/models"}

## Training Model

In [ ]:
dense_layers = [3]
conv_layers = [1]
no_of_filters = [64]
layer_sizes = [128]
dropouts = [0.2]
batch_sizes = [256]
epochs = [300]


for dense_layer in dense_layers:
    for conv_layer in conv_layers:
        for filters in no_of_filters:
            for layer_size in layer_sizes:
                for dropout in dropouts:
                    for batch_size in batch_sizes:
                        for epoch in epochs:

                            model = Sequential()

                            model.add(Conv1D(filters=filters, kernel_size=5, input_shape=(26,1)))
                            model.add(Activation('relu'))
                            model.add(Dropout(dropout))
                            model.add(MaxPooling1D(pool_size=2, padding='same'))

                            for _ in range(conv_layer-1):
                                model.add(Conv1D(filters=filters, kernel_size=3))
                                model.add(Activation('relu'))
                                model.add(Dropout(dropout))
                                model.add(MaxPooling1D(pool_size=2, padding='same'))

                            model.add(Flatten())

                            for _ in range(dense_layer):
                                model.add(Dense(layer_size))
                                model.add(Activation('relu'))
                                model.add(Dropout(dropout))

                            model.add(Dense(10))
                            model.add(Activation('softmax'))

                            model.compile(optimizer='adam',
                                  loss='sparse_categorical_crossentropy',
                                  metrics=['accuracy'])

                            name = f"FEA_EXT_DENSE_{dense_layer}DLay_{conv_layer}CLay_{filters}Flt_{layer_size}LSz_{dropout}DO_{int(time.time())}"
                            filepath = f"{paths['checkpath']}/{name}.best.hdf5"

                            #tensorboard = TensorBoard(log_dir=f"C:\\Users\\Lder\\Documents\\ML_PROJ\\log\\fincnn15\\{NAME}")

                            #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50)
                            checkpointlist = [ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max'),
                                              ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')]

                            for i in range(len(checkpointlist)):
                                hist = model.fit(X, 
                                                 y,
                                                 batch_size = batch_size,
                                                 epochs = epoch,
                                                 callbacks = [checkpointlist[i]],
                                                 validation_split = 0.2 )

                                val_acc = hist.history['val_accuracy']

                                max_val_acc = max(val_acc)

                                max_val_acc_loss = hist.history['val_loss'][val_acc.index(max(val_acc))]
                                
                                if (i==0):
                                    val_acc = hist.history['val_accuracy']
                                    max_val_acc = max(val_acc)
                                    max_val_acc_loss = hist.history['val_loss'][val_acc.index(max(val_acc))]
                                    model_name = f"{max_val_acc:.4f}acc_{max_val_acc_loss:.4f}loss_{dropout}do_{batch_size}bs"
                                elif (i==1):
                                    val_loss = hist.history['val_loss']
                                    min_val_loss = min(val_loss)
                                    min_val_loss_acc = hist.history['val_accuracy'][val_loss.index(min(val_loss))]
                                    model_name = f"{min_val_loss:.4f}loss_{min_val_loss_acc:.4f}acc_{dropout}do_{batch_size}bs"

                                model.load_weights(f"{paths['checkpath']}/{name}.best.hdf5")
                                
                                if (i==0):
                                    model.save(f"{paths['modelpath']}/acc/{model_name}")
                                elif (i==1):
                                    model.save(f"{paths['modelpath']}/loss/{model_name}")

In [ ]:
model = tf.keras.models.load_model(f"{paths['modelpath']}/0.7760acc_0.7978loss")

In [ ]:
model.summary()